# Validate Extracted Sequences

> "Sequences we extracted could have introns in them. Lets validate that DNA maps to protein as expected before we train."

In [ ]:
#| default_exp features.validation

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from Bio import SeqIO

/home/jdb/projects/llm-mito-scanner/venv/lib/python3.10/site-packages/Bio/__init__.py:138: BiopythonWarning: You may be importing Biopython from inside the source tree. This is bad practice and might lead to downstream issues. In particular, you might encounter ImportErrors due to missing compiled C extensions. We recommend that you try running your code from outside the source tree. If you are outside the source tree then you have a setup.py file in an unexpected directory: /home/jdb/projects/llm-mito-scanner/venv/lib/python3.10/site-packages
  warnings.warn(


In [ ]:
#| hide
from yaml import safe_load

tqdm.pandas(ncols=80, leave=False)

with open("../config.yml") as f:
    config = safe_load(f)

In [ ]:
#| hide
import json

data_path = Path(config.get("data_path"))
annotations_path = data_path / "annotations"
genome_path = data_path / "genome"
sequences_path = data_path / "entrez_genes_sequences"
protein_path = data_path / "protein"

with (data_path / "entrez_gene_to_protein_map.json").open("r") as f:
    gene_to_protein_map = json.load(f)
    
len(gene_to_protein_map)

20080

In [ ]:
#| export
def load_gene_and_protein(
    gene_id: str, 
    protein_id: str, 
    gene_path: Path, 
    protein_path: Path
) -> (SeqIO.SeqRecord, SeqIO.SeqRecord):
    gene = next(SeqIO.parse(gene_path / f"{gene_id}.fasta", "fasta"))
    protein = next(SeqIO.parse(protein_path / f"{protein_id}.fasta", "fasta"))
    return gene, protein

In [ ]:
#| hide
gene_ids = list(gene_to_protein_map.keys())
test_gene_id = gene_ids[0]
test_protein_id = gene_to_protein_map.get(test_gene_id)
test_gene_id, test_protein_id

('79501', 'NP_001005484.2')

In [ ]:
#| hide
test_gene, test_protein = load_gene_and_protein(
    test_gene_id,
    test_protein_id,
    sequences_path,
    protein_path
)

In [ ]:
#| export
def validate_gene_and_protein(gene: SeqIO.SeqRecord, protein: SeqIO.SeqRecord) -> bool:
    """
    Returns true if a dna sequence maps directly to a protein sequence.
    """
    translated_gene = gene.translate(cds=True)
    return translated_gene.seq == protein.seq

In [ ]:
#| hide
validate_gene_and_protein(test_gene, test_protein)

TranslationError: First codon 'CCA' is not a start codon

In [ ]:
test_gene.translate().seq

Seq('PDLFRYI*SIHKGLLINQVVY*KGQFTTYYSLTVFMPHSVKIAVVSSSYEEGRW...SKK')

In [ ]:
test_gene.seq in test_protein.seq

False